[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//saeidtafazzol/Indirect-Method-for-Low-Thrust-Trajectory-Design-in-Python-/blob/main/code.ipynb)


# Space Craft Dynamics

In this section, we will talk about the dynamics of the spacecraft assuming that it is only affected by the gravity field of **one** central body.

Let $\boldsymbol{r} = [x,y,z]^\top$ and $\boldsymbol{v} = [v_x,v_y,v_z]^\top$ denote the position and velocity vectors of the center of mass of the spacecraft relative to the origin of the heliocentric frame of reference. The dynamics can be written as:

$$
\begin{align}
\dot{\boldsymbol{r}} &= \boldsymbol{v}, \nonumber \\
\dot{\boldsymbol{v}} &= -\dfrac{\mu}{r^3} \boldsymbol{r} + \dfrac{T_{\max}}{m} \hat{\boldsymbol{\alpha}}  \delta, \\
\dot{m} &= - \dfrac{T_{\max}}{c} \delta
\end{align}
$$

### Explanation of the Dynamics:

1. **Position Update:**

   $$
   \dot{\boldsymbol{r}} = \boldsymbol{v}
   $$

   The velocity vector $\boldsymbol{v}$ represents the rate of change of position $\boldsymbol{r}$.

2. **Velocity Update (Equation of Motion):**

   $$
   \dot{\boldsymbol{v}} = -\dfrac{\mu}{r^3} \boldsymbol{r} + \dfrac{T_{\max}}{m} \hat{\boldsymbol{\alpha}}  \delta
   $$

   - The term $-\dfrac{\mu}{r^3} \boldsymbol{r}$ accounts for the gravitational acceleration due to the central body.
   - The term $\dfrac{T_{\max}}{m} \hat{\boldsymbol{\alpha}}  \delta$ represents the acceleration from an external thrust force, where $\boldsymbol{\alpha}$ is the direction of the thrust and $\delta$ is the amount of propulsion.

3. **Mass Change Due to Propellant Use:**

   $$
   \dot{m} = - \dfrac{T_{\max}}{c} \delta
   $$

   The spacecraft loses mass over time due to fuel consumption, where $c$ is the exhaust velocity of the propellant.

These equations describe the motion of a spacecraft under the influence of a single gravitational body and an external thrust force.



In [4]:
import sympy as sp
import numpy as np

mu = sp.Symbol('mu')
T_max = sp.Symbol('T_{max}')
c = sp.Symbol('c')

r0,r1,r2 = sp.symbols('r0:3')
v0,v1,v2 = sp.symbols('v0:3')
m = sp.Symbol('m')

r = sp.Matrix(3,1,[r0,r1,r2])
v = sp.Matrix(3,1,[v0,v1,v2])

states = sp.Matrix([r,v,m] )
costates = sp.Matrix(7, 1, sp.symbols('lambda0:7'))

len_r = sp.sqrt((r.T@r)[0])

alpha = sp.Matrix(3, 1, sp.symbols('alpha0:3'))
delta = sp.Symbol('delta')

state_dot = sp.Matrix([
v,
-mu/(len_r**3)*r + delta*T_max/m*alpha,
-T_max/c*delta,
])

print('Dynamics:')
state_dot

Dynamics:


Matrix([
[                                                           v0],
[                                                           v1],
[                                                           v2],
[T_{max}*alpha0*delta/m - mu*r0/(r0**2 + r1**2 + r2**2)**(3/2)],
[T_{max}*alpha1*delta/m - mu*r1/(r0**2 + r1**2 + r2**2)**(3/2)],
[T_{max}*alpha2*delta/m - mu*r2/(r0**2 + r1**2 + r2**2)**(3/2)],
[                                             -T_{max}*delta/c]])

# Mission

In this example, we want to start the mission at a certain time from Earth and end at a certain time on Mars. Therefore, we should control the spacecraft so that it fulfills our boundary conditions. I will talk about how later, but for now, let's just set up our variables:

$$
\begin{aligned}
&\bm{r}(t_0)=  \bm{r}_0,\\
&\bm{v}(t_0) = \bm{v}_0 ,\\
& m(t_0) = m_0,\\
&\bm{r}(t_f) - \bm{r}_f = \bm{0},\\
&\bm{v}(t_f) - \bm{v}_f = \bm{0}
\end{aligned}
$$

The discrepancy between the initial and final time conditions is deliberate, and we'll get to it later. (Hint: error function!)

Another important aspect of these missions is normalization. Here, we literally deal with astronomical units, and different variables do not necessarily have the same scale. To balance this, we also normalize our parameters.

In [ ]:
AU = 1.496e+8 # km
AUm = AU*1e3 # m
TU = 31536000/(2*np.pi) #s
mu = 132712440018 / (AU**3)*(TU**2) # gravitational parameter
M = 1000 #kg

m_0 = 1000/M
g0 = 9.8065 # m/s^2
I_sp = 2000 

T_max = 0.5 / AUm * (TU**2) / M
c = I_sp*g0 /AUm * TU

r_0 = np.array([-140699693, -51614428, 980])/AU # Earth's position
v_0 = np.array([9.774596, -28.07828 , 4.337725e-4]) /AU * TU # Earth's velocity

r_f = np.array([-172682023, 176959469, 7948912])/AU # Mars' position
v_f = np.array([-16.427384, -14.860506, 9.21486e-2])/AU*TU # Mars' velocity

t_f = 348.795  * 24 * 3600 / TU # time of flight

# Objective

For efficiency, we need to design trajectories that consume as little fuel as possible, so we introduce this objective:

$$J =  \int_{t_0}^{t_f} \dfrac{T_{\max}}{c}\delta(t) \, dt$$

which basically adds up all the fuel we use throughout the trajectory. Now using what's called **Indirect Method**, we can find a **continious expression** for the optimal control we are after. Although, it's beyond the scope of this tutorial to go through the detailed derivation, I will try to provide explanations that you have to take with a grain of salt.

To optimize for the objective, we take a very similar approach as lagrange multipliers (in multivariates calculus). You may ask, but the objective is not constrained, but it is. It is constrained to the dynamics previously introduced. Similar, 